In [1]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
import folium
from math import sqrt
from shapely import wkt
import json
from shapely.geometry import Polygon, LineString
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)
from popframe.method.popuation_frame import PopFrame

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Помощь

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Определение функции для поиска и удаления дубликатов
def find_and_remove_duplicates(gdf, name_column, geometry_column, population_column):
    tolerance_km = 40
    tolerance_deg = tolerance_km / 111  # Преобразование километров в градусы (приблизительно)
    
    # Создание пространственного индекса
    gdf = gdf.reset_index(drop=True)
    gdf['duplicate'] = False
    
    spatial_index = gdf.sindex
    
    for idx, row in gdf.iterrows():
        if gdf.at[idx, 'duplicate']:
            continue
        
        # Поиск ближайших точек в пределах допустимого расстояния
        possible_matches_index = list(spatial_index.intersection(row[geometry_column].buffer(tolerance_deg).bounds))
        possible_matches = gdf.iloc[possible_matches_index]
        exact_matches = possible_matches[possible_matches[name_column] == row[name_column]]
        exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]
        
        # Проверка разницы в населении
        exact_matches = exact_matches[exact_matches[population_column].apply(lambda x: x <= row[population_column] * 1.5 and x >= row[population_column] / 1.5)]
        
        if len(exact_matches) > 1:
            duplicate_indices = exact_matches.index
            # Помечаем все, кроме одного, как дубликаты
            gdf.loc[duplicate_indices[1:], 'duplicate'] = True

    return gdf

In [3]:
from popframe.models.region import Region
region_model = Region.from_pickle('C:\Code\PopFrame\examples\data\model_data/region_noter.pickle')
towns = region_model.get_towns_gdf()
towns

,id,name,population,level,geometry,settlement_name,district_name
0,0,Болото,111,Малое сельское поселение,POINT (543142.339 6580637.744),Борское сельское поселение,Бокситогорский муниципальный район
1,1,Большой Остров,116,Малое сельское поселение,POINT (544532.929 6593228.214),Борское сельское поселение,Бокситогорский муниципальный район
2,2,Бор,116,Малое сельское поселение,POINT (544895.402 6593052.876),Борское сельское поселение,Бокситогорский муниципальный район
3,3,Бороватое,114,Малое сельское поселение,POINT (543993.111 6589582.121),Борское сельское поселение,Бокситогорский муниципальный район
4,4,Бочево,108,Малое сельское поселение,POINT (538540.110 6576793.684),Борское сельское поселение,Бокситогорский муниципальный район
...,...,...,...,...,...,...,...
2926,2926,Апраксин Бор,96,Малое сельское поселение,POINT (399059.742 6560341.471),Трубникоборское сельское поселение,Тоснненский муниципальный район
2927,2927,Александровка,98,Малое сельское поселение,POINT (403967.171 6562087.081),Трубникоборское сельское поселение,Тоснненский муниципальный район
2928,2928,Большая Горка,100,Малое сельское поселение,POINT (413116.740 6573793.279),Трубникоборское сельское поселение,Тоснненский муниципальный район
2929,2929,Дроздово,101,Малое сельское поселение,POINT (412458.032 6574811.307),Трубникоборское сельское поселение,Тоснненский муниципальный район


In [4]:
visualizer = PopFrame(region=region_model)
output = visualizer.save_output(output_type='html')  # or 'gdf'
output

'HTML map saved as final_square_LO.html'

In [5]:
visualizer = PopFrame(region=region_model)
output = visualizer.save_output(output_type='gdf')  # or 'gdf'
output

,id,name,population,level,geometry,settlement_name,district_name
0,0,Болото,111,Малое сельское поселение,"POLYGON ((33.75629 59.35963, 33.76156 59.35963...",Борское сельское поселение,Бокситогорский муниципальный район
1,1,Большой Остров,116,Малое сельское поселение,"POLYGON ((33.78330 59.47248, 33.78869 59.47248...",Борское сельское поселение,Бокситогорский муниципальный район
2,2,Бор,116,Малое сельское поселение,"POLYGON ((33.78966 59.47087, 33.79505 59.47087...",Борское сельское поселение,Бокситогорский муниципальный район
3,3,Бороватое,114,Малое сельское поселение,"POLYGON ((33.77305 59.43982, 33.77839 59.43982...",Борское сельское поселение,Бокситогорский муниципальный район
4,4,Бочево,108,Малое сельское поселение,"POLYGON ((33.67468 59.32560, 33.67988 59.32560...",Борское сельское поселение,Бокситогорский муниципальный район
...,...,...,...,...,...,...,...
2926,2926,Апраксин Бор,96,Малое сельское поселение,"POLYGON ((31.23176 59.16776, 31.23665 59.16776...",Трубникоборское сельское поселение,Тоснненский муниципальный район
2927,2927,Александровка,98,Малое сельское поселение,"POLYGON ((31.31677 59.18455, 31.32172 59.18455...",Трубникоборское сельское поселение,Тоснненский муниципальный район
2928,2928,Большая Горка,100,Малое сельское поселение,"POLYGON ((31.47212 59.29158, 31.47712 59.29158...",Трубникоборское сельское поселение,Тоснненский муниципальный район
2929,2929,Дроздово,101,Малое сельское поселение,"POLYGON ((31.46014 59.30057, 31.46516 59.30057...",Трубникоборское сельское поселение,Тоснненский муниципальный район


In [6]:
output.explore()

In [6]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
import matplotlib.colors as mcolors
import numpy as np

towns = region_model.get_towns_gdf()


# Функция для создания квадратного полигона вокруг точки
def create_square(center, size):
    half_size = size / 2.0
    return Polygon([
        (center.x - half_size, center.y - half_size),
        (center.x + half_size, center.y - half_size),
        (center.x + half_size, center.y + half_size),
        (center.x - half_size, center.y + half_size)
    ])

# Определяем размеры квадратов на основе населения
def size_from_population(population, level):
    if level in ["Малое сельское поселение", "Среднее сельское поселение", "Большое сельское поселение", "Крупное сельское поселение"]:
        return 0.0005 * (population ** 0.5)   # Логарифмическая шкала для малых населенных пунктов
    elif level == "Сверхкрупный город":
        return 0.0001 * (population ** 0.5)  # Уменьшенная линейная шкала для сверхкрупных городов
    return 0.0002 * (population ** 0.5)  # Линейная шкала для крупных населенных пунктов

# Создание квадратов
def convert_points_to_squares(gdf):
    new_geometries = []
    for idx, row in gdf.iterrows():
        if isinstance(row['geometry'], Point):
            size = size_from_population(row['population'], row['level'])
            square = create_square(row['geometry'], size)
            new_geometries.append(square)
        else:
            new_geometries.append(row['geometry'])
    gdf['geometry'] = new_geometries
    return gdf

gdf = convert_points_to_squares(gdf)

# Функция для генерации цветовой карты
def get_color_map(levels):
    base_colors = ['#FFC100', '#FF6500', '#C40C0C', '#6C0345']
    cmap = mcolors.LinearSegmentedColormap.from_list("heatmap", base_colors, N=len(levels))
    norm = mcolors.Normalize(vmin=0, vmax=len(levels) - 1)
    colors = [mcolors.to_hex(cmap(norm(i))) for i in range(len(levels))]
    return colors

# Получение цветов для уровней
levels = list(gdf['level'].unique())
levels.sort(key=lambda x: ["Малое сельское поселение", "Среднее сельское поселение", "Большое сельское поселение", "Крупное сельское поселение", "Малый город", "Средний город", "Большой город", "Крупный город", "Крупнейший город", "Сверхкрупный город"].index(x))
level_colors = dict(zip(levels, get_color_map(levels)))

# Визуализация на карте
m = folium.Map(zoom_start=10, tiles='CartoDB Positron')

for _, row in gdf.iterrows():
    color = level_colors[row['level']]
    popup_text = f"{row['name']}<br>Размер: {row['level']}<br>Население: {row['population']}"
    
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 2,
            'fillOpacity': 0.6
        },
        tooltip=popup_text
    ).add_to(m)

# Создание легенды
legend_html = '<div style="position: fixed; bottom: 50px; left: 50px; width: 300px; height: 350px; border:2px solid grey; z-index:9999; font-size:14px; background-color:white; padding: 10px;"><b>Легенда</b><br>'
for level, color in level_colors.items():
    legend_html += f'&nbsp; {level} &nbsp; <i class="fa fa-square" style="font-size:20px;color:{color}"></i><br>'
legend_html += '</div>'

# Добавление легенды на карту
m.get_root().html.add_child(folium.Element(legend_html))

# Сохранение карты в HTML-файл
m.save('final_square_LO.html')
# gdf.to_file("gdf_square_LO.geojson", driver="GeoJSON")

## Тюмень

In [3]:
towns = gpd.read_file('data_9/Тюменская область/towns.geojson')
okrugs = gpd.read_file('data_9/Тюменская область/region.geojson')
rayons = gpd.read_file('/Users/mvin/Code/PopFrame/data_9/Тюменская область/districts.geojson')
adj_mx = pd.read_pickle("data_9/Тюменская область/adj_mx.pickle")


In [4]:
towns

,name,population,geometry
0,Красивое,192,POINT (70.15830 55.20878)
1,Александровка,629,POINT (69.94983 55.26997)
2,Михайловка,184,POINT (70.08474 55.26131)
3,Покровка,240,POINT (70.09869 55.43390)
4,Таволжан,181,POINT (70.15640 55.34947)
...,...,...,...
1232,Новоказанка,22,POINT (65.27978 57.44325)
1233,Новопокровка,361,POINT (65.37807 57.47281)
1234,Ахманы,104,POINT (64.97423 57.53028)
1235,Средние Тарманы,489,POINT (65.36232 57.36702)


In [5]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (70.15830 55.20878),Красивое,192,Малое сельское поселение
1,POINT (69.94983 55.26997),Александровка,629,Среднее сельское поселение
2,POINT (70.08474 55.26131),Михайловка,184,Малое сельское поселение
3,POINT (70.09869 55.43390),Покровка,240,Среднее сельское поселение
4,POINT (70.15640 55.34947),Таволжан,181,Малое сельское поселение
...,...,...,...,...
1232,POINT (65.27978 57.44325),Новоказанка,22,Малое сельское поселение
1233,POINT (65.37807 57.47281),Новопокровка,361,Среднее сельское поселение
1234,POINT (64.97423 57.53028),Ахманы,104,Малое сельское поселение
1235,POINT (65.36232 57.36702),Средние Тарманы,489,Среднее сельское поселение


In [6]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=okrugs, 
  districts=rayons, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [7]:
# region.to_pickle('72.pickle')

In [ ]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (70.15830 55.20878),Красивое,Малое сельское поселение,192.0
1,POINT (69.94983 55.26997),Александровка,Среднее сельское поселение,629.0
2,POINT (70.08474 55.26131),Михайловка,Малое сельское поселение,184.0
3,POINT (70.09869 55.43390),Покровка,Среднее сельское поселение,240.0
4,POINT (70.15640 55.34947),Таволжан,Малое сельское поселение,181.0
...,...,...,...,...
2468,"LINESTRING (65.22014 57.28574, 65.01014 57.37856)",NaN,Большое сельское поселение,NaN
2469,"LINESTRING (65.30903 57.53569, 65.16233 57.50669)",NaN,Малое сельское поселение,NaN
2470,"LINESTRING (65.16233 57.50669, 65.16725 57.53747)",NaN,Малое сельское поселение,NaN
2471,"LINESTRING (65.16233 57.50669, 65.27978 57.44325)",NaN,Малое сельское поселение,NaN


In [ ]:
# gdf_frame.to_file("gdf_frame_Tumen.geojson", driver="GeoJSON")
# frame_method.get_graph_html(G, 'final_graph_Tumen.html')

## Московская область

In [8]:
towns = gpd.read_parquet('data_9/1111/towns.parquet')
region = gpd.read_file('data_9/1111/region.geojson')
districts = gpd.read_file('data_9/1111\districts.geojson')
# adj_mx = pd.read_pickle("data_9/1111/adj_mx.pickle")
graph = nx.read_graphml('data_9/1111/graph.graphml')

FileNotFoundError: data_9/1111/towns.parquet

In [ ]:
# # Применение функции для поиска и удаления дубликатов
# gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# # Удаление дубликатов из исходного GeoDataFrame
# towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

# towns.reset_index(drop=True, inplace=True)
# towns.to_parquet('towns.parquet', engine='pyarrow')
# towns

In [ ]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.0001)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2755379623.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [ ]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (38.72237 54.46798),Серебряные Пруды,9358,Малый город
1,POINT (38.61833 54.54061),Узуново,3417,Крупное сельское поселение
2,POINT (38.62932 54.47175),Благодать,52,Малое сельское поселение
3,POINT (38.07726 54.88660),Ступино,66463,Средний город
4,POINT (38.15051 54.83745),Кашира,39929,Малый город
...,...,...,...,...
2873,POINT (35.60816 56.37178),Шелгуново,9,Малое сельское поселение
2874,POINT (35.68766 56.36313),Мазлово,2,Малое сельское поселение
2875,POINT (35.52194 56.37250),Себудово,2,Малое сельское поселение
2876,POINT (35.63585 56.37377),Введенское,694,Среднее сельское поселение


In [ ]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [ ]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (38.72237 54.46798),Серебряные Пруды,Малый город,9358.0
1,POINT (38.61833 54.54061),Узуново,Крупное сельское поселение,3417.0
2,POINT (38.62932 54.47175),Благодать,Малое сельское поселение,52.0
3,POINT (38.07726 54.88660),Ступино,Средний город,66463.0
4,POINT (38.15051 54.83745),Кашира,Малый город,39929.0
...,...,...,...,...
5750,"LINESTRING (35.50998 56.36097, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN
5751,"LINESTRING (35.62201 56.37238, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN
5752,"LINESTRING (35.60816 56.37178, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN
5753,"LINESTRING (35.68766 56.36313, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN


In [ ]:
gdf_frame.to_file("gdf_frame_MO.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_MO.html')

## Краснодар

In [ ]:
towns = gpd.read_parquet('data_9/2222/towns.parquet')
# towns = gpd.read_file('data_9/2222/towns.geojson')
region = gpd.read_file('data_9/2222/region.geojson')
districts = gpd.read_file('data_9/2222\districts.geojson')
# adj_mx = pd.read_pickle("data_9/1111/adj_mx.pickle")
graph = nx.read_graphml('data_9/2222/graph.graphml')

In [ ]:
towns

,name,population,geometry
0,Сочи,446599,POINT (39.72311 43.58548)
1,Дагомыс,17841,POINT (39.65444 43.65900)
2,Солохаул,214,POINT (39.67738 43.80080)
3,Пластунка,2270,POINT (39.76183 43.66878)
4,Нижняя Шиловка,5432,POINT (40.02332 43.45000)
...,...,...,...
1597,Солёный,486,POINT (37.03120 45.26675)
1598,Артющенко,90,POINT (36.81234 45.13542)
1599,Виноградный,1988,POINT (36.89917 45.19328)
1600,Пересыпь,776,POINT (37.13355 45.34775)


In [ ]:
# Применение функции для поиска и удаления дубликатов
gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# Удаление дубликатов из исходного GeoDataFrame
towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

towns.reset_index(drop=True, inplace=True)
# towns.to_parquet('towns.parquet', engine='pyarrow')
# towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]


In [ ]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.0001)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2755379623.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [ ]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (39.72311 43.58548),Сочи,446599,Крупный город
1,POINT (39.65444 43.65900),Дагомыс,17841,Малый город
2,POINT (39.67738 43.80080),Солохаул,214,Среднее сельское поселение
3,POINT (39.76183 43.66878),Пластунка,2270,Большое сельское поселение
4,POINT (40.02332 43.45000),Нижняя Шиловка,5432,Малый город
...,...,...,...,...
1597,POINT (37.03120 45.26675),Солёный,486,Среднее сельское поселение
1598,POINT (36.81234 45.13542),Артющенко,90,Малое сельское поселение
1599,POINT (36.89917 45.19328),Виноградный,1988,Большое сельское поселение
1600,POINT (37.13355 45.34775),Пересыпь,776,Среднее сельское поселение


In [ ]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [ ]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (39.72311 43.58548),Сочи,Крупный город,446599.0
1,POINT (39.65444 43.65900),Дагомыс,Малый город,17841.0
2,POINT (39.67738 43.80080),Солохаул,Среднее сельское поселение,214.0
3,POINT (39.76183 43.66878),Пластунка,Большое сельское поселение,2270.0
4,POINT (40.02332 43.45000),Нижняя Шиловка,Малый город,5432.0
...,...,...,...,...
3198,"LINESTRING (36.82488 45.39271, 36.86435 45.39810)",NaN,Среднее сельское поселение,NaN
3199,"LINESTRING (36.93308 45.36573, 36.94821 45.33243)",NaN,Среднее сельское поселение,NaN
3200,"LINESTRING (36.98520 45.33221, 37.03120 45.26675)",NaN,Среднее сельское поселение,NaN
3201,"LINESTRING (37.10490 45.32134, 37.13355 45.34775)",NaN,Среднее сельское поселение,NaN


In [ ]:
nodes = list(G.nodes(data=True))
isolated_nodes = [node for node in nodes if G.degree(node[0]) == 0]
isolated_nodes

[]

In [ ]:
gdf_frame.to_file("gdf_frame_Krasnodar.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_Krasnodar.html')

## Омская область

In [ ]:
# towns = gpd.read_parquet('data_9/omsk/towns.parquet')
towns = gpd.read_file('data_9/omsk/towns.geojson')
region = gpd.read_file('data_9/omsk/region.geojson')
districts = gpd.read_file('data_9/omsk\districts.geojson')
# adj_mx = pd.read_pickle("data_9/1111/adj_mx.pickle")
graph = nx.read_graphml('data_9/omsk/graph.graphml')

In [ ]:
towns

,name,population,geometry
0,Русская Поляна,6164,POINT (73.88210 53.78036)
1,Пограничное,162,POINT (73.73034 53.83624)
2,Андриановка,210,POINT (73.75246 53.80525)
3,Тогунас,184,POINT (73.55164 53.77518)
4,Новоивановка,6,POINT (73.81005 53.79318)
...,...,...,...
1450,Березянка,49,POINT (70.74768 57.79322)
1451,Затон,3,POINT (70.91183 57.75162)
1452,Саургачи,429,POINT (70.69388 57.76643)
1453,Саургачи,9,POINT (70.71890 57.74734)


In [ ]:
gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# Удаление дубликатов из исходного GeoDataFrame
towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

towns.reset_index(drop=True, inplace=True)
towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]


,name,population,geometry
0,Русская Поляна,6164,POINT (73.88210 53.78036)
1,Пограничное,162,POINT (73.73034 53.83624)
2,Андриановка,210,POINT (73.75246 53.80525)
3,Тогунас,184,POINT (73.55164 53.77518)
4,Новоивановка,6,POINT (73.81005 53.79318)
...,...,...,...
1449,Березянка,49,POINT (70.74768 57.79322)
1450,Затон,3,POINT (70.91183 57.75162)
1451,Саургачи,429,POINT (70.69388 57.76643)
1452,Саургачи,9,POINT (70.71890 57.74734)


In [ ]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.0001)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2755379623.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [ ]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (73.88210 53.78036),Русская Поляна,6164,Малый город
1,POINT (73.73034 53.83624),Пограничное,162,Малое сельское поселение
2,POINT (73.75246 53.80525),Андриановка,210,Среднее сельское поселение
3,POINT (73.55164 53.77518),Тогунас,184,Малое сельское поселение
4,POINT (73.81005 53.79318),Новоивановка,6,Малое сельское поселение
...,...,...,...,...
1449,POINT (70.74768 57.79322),Березянка,49,Малое сельское поселение
1450,POINT (70.91183 57.75162),Затон,3,Малое сельское поселение
1451,POINT (70.69388 57.76643),Саургачи,429,Среднее сельское поселение
1452,POINT (70.71890 57.74734),Саургачи,9,Малое сельское поселение


In [ ]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [ ]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (73.88210 53.78036),Русская Поляна,Малый город,6164.0
1,POINT (73.73034 53.83624),Пограничное,Малое сельское поселение,162.0
2,POINT (73.75246 53.80525),Андриановка,Среднее сельское поселение,210.0
3,POINT (73.55164 53.77518),Тогунас,Малое сельское поселение,184.0
4,POINT (73.81005 53.79318),Новоивановка,Малое сельское поселение,6.0
...,...,...,...,...
2902,"LINESTRING (70.69189 57.29081, 70.73530 57.35387)",NaN,Малое сельское поселение,NaN
2903,"LINESTRING (70.49588 57.91413, 70.45632 57.93720)",NaN,Малое сельское поселение,NaN
2904,"LINESTRING (70.43469 57.92085, 70.45632 57.93720)",NaN,Малое сельское поселение,NaN
2905,"LINESTRING (70.45632 57.93720, 70.71890 57.74734)",NaN,Малое сельское поселение,NaN


In [ ]:
gdf_frame.to_file("gdf_frame_Omsk.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_Omsk.html')

## Тульская область

In [ ]:
# towns = gpd.read_parquet('data_9/omsk/towns.parquet')
towns = gpd.read_file('data_9/tulsk/towns.geojson')
region = gpd.read_file('data_9/tulsk/region.geojson')
districts = gpd.read_file('data_9/tulsk\districts.geojson')
# adj_mx = pd.read_pickle("data_9/tulsk/adj_mx.pickle")
graph = nx.read_graphml('data_9/tulsk/graph.graphml')

In [ ]:
towns
gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# Удаление дубликатов из исходного GeoDataFrame
towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

towns.reset_index(drop=True, inplace=True)
towns
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.0001)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]
C:\Users\max\AppData\Local\Temp\ipykernel_39084\3104660453.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,geometry,name,population,level
0,POINT (38.11619 53.14183),Ефремов,36181,Малый город
1,POINT (37.94581 53.15550),Круглое,153,Малое сельское поселение
2,POINT (38.00600 53.55480),Волово,3563,Крупное сельское поселение
3,POINT (37.81342 53.99528),Бородинский,6020,Малый город
4,POINT (38.12696 53.77357),Богородицк,31363,Малый город
...,...,...,...,...
210,POINT (36.33867 54.11014),Курорт Краинка,576,Среднее сельское поселение
211,POINT (36.47392 54.15941),Агеево,501,Среднее сельское поселение
212,POINT (36.34253 54.10175),Новая Черепеть,499,Среднее сельское поселение
213,POINT (36.32409 54.10941),Рождествено,886,Среднее сельское поселение


In [ ]:

from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame
gdf_frame.to_file("gdf_frame_Tulsk.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_Tulsk.html')

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]
C:\Users\max\AppData\Local\Temp\ipykernel_39084\1925723099.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


## Волгоградская область

In [ ]:
# towns = gpd.read_parquet('data_9/omsk/towns.parquet')
towns = gpd.read_file('data_9/volga/towns.geojson')
region = gpd.read_file('data_9/volga/region.geojson')
districts = gpd.read_file('data_9/volga\districts.geojson')
# adj_mx = pd.read_pickle("data_9/tulsk/adj_mx.pickle")
graph = nx.read_graphml('data_9/volga/graph.graphml')

In [ ]:
towns

,name,population,geometry
0,Пимено-Черни,1088,POINT (43.47043 47.70139)
1,Черноморовский,210,POINT (43.43179 48.24480)
2,Громославка,820,POINT (43.62032 48.19695)
3,Васильевка,500,POINT (43.81995 48.13542)
4,Водино,180,POINT (43.98373 48.01802)
...,...,...,...
1207,Меловский,80,POINT (41.79228 50.49351)
1208,Тушкановский,310,POINT (41.82125 50.52874)
1209,Соколовский,270,POINT (41.84307 50.38606)
1210,Павловский,770,POINT (41.79589 50.39507)


In [ ]:
gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# Удаление дубликатов из исходного GeoDataFrame
towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

towns.reset_index(drop=True, inplace=True)
towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]


,name,population,geometry
0,Пимено-Черни,1088,POINT (43.47043 47.70139)
1,Черноморовский,210,POINT (43.43179 48.24480)
2,Громославка,820,POINT (43.62032 48.19695)
3,Васильевка,500,POINT (43.81995 48.13542)
4,Водино,180,POINT (43.98373 48.01802)
...,...,...,...
1200,Меловский,80,POINT (41.79228 50.49351)
1201,Тушкановский,310,POINT (41.82125 50.52874)
1202,Соколовский,270,POINT (41.84307 50.38606)
1203,Павловский,770,POINT (41.79589 50.39507)


In [ ]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.0001)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2755379623.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [ ]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (43.47043 47.70139),Пимено-Черни,1088,Большое сельское поселение
1,POINT (43.43179 48.24480),Черноморовский,210,Среднее сельское поселение
2,POINT (43.62032 48.19695),Громославка,820,Среднее сельское поселение
3,POINT (43.81995 48.13542),Васильевка,500,Среднее сельское поселение
4,POINT (43.98373 48.01802),Водино,180,Малое сельское поселение
...,...,...,...,...
1200,POINT (41.79228 50.49351),Меловский,80,Малое сельское поселение
1201,POINT (41.82125 50.52874),Тушкановский,310,Среднее сельское поселение
1202,POINT (41.84307 50.38606),Соколовский,270,Среднее сельское поселение
1203,POINT (41.79589 50.39507),Павловский,770,Среднее сельское поселение


In [ ]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [ ]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (43.47043 47.70139),Пимено-Черни,Большое сельское поселение,1088.0
1,POINT (43.43179 48.24480),Черноморовский,Среднее сельское поселение,210.0
2,POINT (43.62032 48.19695),Громославка,Среднее сельское поселение,820.0
3,POINT (43.81995 48.13542),Васильевка,Среднее сельское поселение,500.0
4,POINT (43.98373 48.01802),Водино,Малое сельское поселение,180.0
...,...,...,...,...
2404,"LINESTRING (41.59877 50.45201, 41.58663 50.41372)",NaN,Малое сельское поселение,NaN
2405,"LINESTRING (41.52641 50.43355, 41.60977 50.39684)",NaN,Малое сельское поселение,NaN
2406,"LINESTRING (41.50520 50.66101, 41.48091 50.63861)",NaN,Малое сельское поселение,NaN
2407,"LINESTRING (41.79228 50.49351, 41.82125 50.52874)",NaN,Малое сельское поселение,NaN


In [ ]:
gdf_frame.to_file("gdf_frame_Volga.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_Volga.html')

## Москва

In [ ]:
# towns = gpd.read_parquet('data_9/omsk/towns.parquet')
towns = gpd.read_file('data_9/moskva/towns.geojson')
region = gpd.read_file('data_9/moskva/region.geojson')
districts = gpd.read_file('data_9/moskva\districts.geojson')
# adj_mx = pd.read_pickle("data_9/tulsk/adj_mx.pickle")
graph = nx.read_graphml('data_9/moskva/graph.graphml')
towns


,name,population,geometry
0,Вороново,301,POINT (37.15539 55.31987)
1,Чириково,37,POINT (37.22963 55.39261)
2,Киевский,13564,POINT (36.86842 55.43026)
3,Яковлевское,4330,POINT (36.94024 55.46415)
4,Исаково,73,POINT (37.18720 55.40600)
...,...,...,...
202,Алхимово,11,POINT (37.50805 55.47582)
203,Сальково,13,POINT (37.52516 55.44151)
204,Городище,124,POINT (37.42779 55.53655)
205,Сосенки,30651,POINT (37.44086 55.56633)


In [ ]:
gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# Удаление дубликатов из исходного GeoDataFrame
towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

towns.reset_index(drop=True, inplace=True)
towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]


,name,population,geometry
0,Вороново,301,POINT (37.15539 55.31987)
1,Киевский,13564,POINT (36.86842 55.43026)
2,Яковлевское,4330,POINT (36.94024 55.46415)
3,Исаково,73,POINT (37.18720 55.40600)
4,Свитино,16,POINT (37.07198 55.32177)
...,...,...,...
195,Алхимово,11,POINT (37.50805 55.47582)
196,Сальково,13,POINT (37.52516 55.44151)
197,Городище,124,POINT (37.42779 55.53655)
198,Сосенки,30651,POINT (37.44086 55.56633)


In [ ]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.0001)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\693140548.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,geometry,name,population,level
0,POINT (37.15539 55.31987),Вороново,301,Среднее сельское поселение
1,POINT (36.86842 55.43026),Киевский,13564,Малый город
2,POINT (36.94024 55.46415),Яковлевское,4330,Крупное сельское поселение
3,POINT (37.18720 55.40600),Исаково,73,Малое сельское поселение
4,POINT (37.07198 55.32177),Свитино,16,Малое сельское поселение
...,...,...,...,...
195,POINT (37.50805 55.47582),Алхимово,11,Малое сельское поселение
196,POINT (37.52516 55.44151),Сальково,13,Малое сельское поселение
197,POINT (37.42779 55.53655),Городище,124,Малое сельское поселение
198,POINT (37.44086 55.56633),Сосенки,30651,Малый город


In [ ]:

from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame
gdf_frame.to_file("gdf_frame_Moscow.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_Moscow.html')

C:\Users\max\AppData\Local\Temp\ipykernel_39084\1117274382.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


## Санкт-Петербург

In [ ]:
# towns = gpd.read_parquet('data_9/omsk/towns.parquet')
towns = gpd.read_file('data_9/SPb/towns.geojson')
region = gpd.read_file('data_9/SPb/region.geojson')
districts = gpd.read_file('data_9/SPb\districts.geojson')
# adj_mx = pd.read_pickle("data_9/tulsk/adj_mx.pickle")
graph = nx.read_graphml('data_9/SPb/graph.graphml')
towns


,name,population,geometry
0,Александровская,2807,POINT (30.34829 59.73138)
1,Пушкин,112978,POINT (30.41573 59.72226)
2,Санкт-Петербург,5597763,POINT (30.31623 59.93873)
3,Колпино,149766,POINT (30.60448 59.74844)
4,Красное Село,60029,POINT (30.08625 59.73377)
5,Сапёрный,1625,POINT (30.69054 59.77800)
6,Понтонный,9273,POINT (30.63096 59.78293)
7,Металлострой,30895,POINT (30.57508 59.80906)
8,Петро-Славянка,1380,POINT (30.51094 59.80319)
9,Усть-Ижора,1982,POINT (30.59951 59.80387)


In [ ]:

gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# Удаление дубликатов из исходного GeoDataFrame
towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

towns.reset_index(drop=True, inplace=True)
towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]


,name,population,geometry
0,Александровская,2807,POINT (30.34829 59.73138)
1,Пушкин,112978,POINT (30.41573 59.72226)
2,Санкт-Петербург,5597763,POINT (30.31623 59.93873)
3,Колпино,149766,POINT (30.60448 59.74844)
4,Красное Село,60029,POINT (30.08625 59.73377)
5,Сапёрный,1625,POINT (30.69054 59.77800)
6,Понтонный,9273,POINT (30.63096 59.78293)
7,Металлострой,30895,POINT (30.57508 59.80906)
8,Петро-Славянка,1380,POINT (30.51094 59.80319)
9,Усть-Ижора,1982,POINT (30.59951 59.80387)


In [ ]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.0001)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

C:\Users\max\AppData\Local\Temp\ipykernel_39084\693140548.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,geometry,name,population,level
0,POINT (30.34829 59.73138),Александровская,2807,Большое сельское поселение
1,POINT (30.41573 59.72226),Пушкин,112978,Большой город
2,POINT (30.31623 59.93873),Санкт-Петербург,5597763,Сверхкрупный город
3,POINT (30.60448 59.74844),Колпино,149766,Большой город
4,POINT (30.08625 59.73377),Красное Село,60029,Средний город
5,POINT (30.69054 59.77800),Сапёрный,1625,Большое сельское поселение
6,POINT (30.63096 59.78293),Понтонный,9273,Малый город
7,POINT (30.57508 59.80906),Металлострой,30895,Малый город
8,POINT (30.51094 59.80319),Петро-Славянка,1380,Большое сельское поселение
9,POINT (30.59951 59.80387),Усть-Ижора,1982,Большое сельское поселение


In [ ]:

from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame
gdf_frame.to_file("gdf_frame_SPb.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_SPb.html')

C:\Users\max\AppData\Local\Temp\ipykernel_39084\1748248714.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.0001)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


## Ленинградская область

In [ ]:
# towns = gpd.read_parquet('data_9/omsk/towns.parquet')
towns = gpd.read_file('data_9/lenobl/towns.geojson')
region = gpd.read_file('data_9/lenobl/region.geojson')
districts = gpd.read_file('data_9/lenobl\districts.geojson')
adj_mx = pd.read_pickle("C:\Code\PopFrame\data_9\lenobl\___adj_mx.pickle")
# graph = nx.read_graphml('data_9/lenobl/graph.graphml')
towns

,name,is_city,median_time,coef,coef_norm,population,geometry
0,Болото,False,1069.8,0.000935,0.032603,111.0,POINT (33.75892 59.36227)
1,Большой Остров,False,1024.6,0.000976,0.034041,116.0,POINT (33.78600 59.47518)
2,Бор,False,1023.4,0.000977,0.034081,116.0,POINT (33.79236 59.47356)
3,Бороватое,False,1039.3,0.000962,0.033560,114.0,POINT (33.77572 59.44249)
4,Бочево,False,1096.4,0.000912,0.031812,108.0,POINT (33.67728 59.32820)
...,...,...,...,...,...,...,...
2926,Апраксин Бор,False,725.9,0.001378,0.059011,96.0,POINT (31.23421 59.17021)
2927,Александровка,False,706.2,0.001416,0.060657,98.0,POINT (31.31924 59.18702)
2928,Большая Горка,False,690.9,0.001447,0.062000,100.0,POINT (31.47462 59.29408)
2929,Дроздово,False,686.1,0.001458,0.062434,101.0,POINT (31.46265 59.30308)


In [ ]:
# gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# # Удаление дубликатов из исходного GeoDataFrame
# towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

# towns.reset_index(drop=True, inplace=True)
# towns

In [ ]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (33.75892 59.36227),Болото,111,Малое сельское поселение
1,POINT (33.78600 59.47518),Большой Остров,116,Малое сельское поселение
2,POINT (33.79236 59.47356),Бор,116,Малое сельское поселение
3,POINT (33.77572 59.44249),Бороватое,114,Малое сельское поселение
4,POINT (33.67728 59.32820),Бочево,108,Малое сельское поселение
...,...,...,...,...
2926,POINT (31.23421 59.17021),Апраксин Бор,96,Малое сельское поселение
2927,POINT (31.31924 59.18702),Александровка,98,Малое сельское поселение
2928,POINT (31.47462 59.29408),Большая Горка,100,Малое сельское поселение
2929,POINT (31.46265 59.30308),Дроздово,101,Малое сельское поселение


In [ ]:
# from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
# blocks = towns.copy()
# blocks.geometry = blocks.geometry.buffer(0.01)
# ac = AdjacencyCalculator(blocks=blocks, graph=graph)
# adj_mx = ac.get_dataframe()

from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame
gdf_frame.to_file("gdf_frame_LO.geojson", driver="GeoJSON")
frame_method.get_graph_html(G, 'final_graph_LO.html')